# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f913cc59640>
├── 'a' --> tensor([[-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748]])
└── 'x' --> <FastTreeValue 0x7f913cc59a00>
    └── 'c' --> tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                        [ 0.2954,  0.3656,  0.1234, -0.7910],
                        [-1.0263,  0.5554,  0.3729,  0.0998]])

In [4]:
t.a

tensor([[-0.3717, -0.9159, -0.9777],
        [-0.9135,  0.7139, -0.1748]])

In [5]:
%timeit t.a

74.5 ns ± 1.44 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f913cc59640>
├── 'a' --> tensor([[-0.4360, -1.8670, -0.7582],
│                   [-0.0349,  1.5001,  1.5903]])
└── 'x' --> <FastTreeValue 0x7f913cc59a00>
    └── 'c' --> tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                        [ 0.2954,  0.3656,  0.1234, -0.7910],
                        [-1.0263,  0.5554,  0.3729,  0.0998]])

In [7]:
%timeit t.a = new_value

91.2 ns ± 0.836 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3717, -0.9159, -0.9777],
               [-0.9135,  0.7139, -0.1748]]),
    x: Batch(
           c: tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998]]),
       ),
)

In [10]:
b.a

tensor([[-0.3717, -0.9159, -0.9777],
        [-0.9135,  0.7139, -0.1748]])

In [11]:
%timeit b.a

71.8 ns ± 0.94 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3792,  1.3291,  0.3455],
               [ 1.4274, -0.1250,  1.2484]]),
    x: Batch(
           c: tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998]]),
       ),
)

In [13]:
%timeit b.a = new_value

615 ns ± 10.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.22 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

306 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

316 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9090a07100>
├── 'a' --> tensor([[[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]],
│           
│                   [[-0.3717, -0.9159, -0.9777],
│                    [-0.9135,  0.7139, -0.1748]]])
└── 'x' --> <FastTreeValue 0x7f9114217670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.1 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9090f4af10>
├── 'a' --> tensor([[-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748],
│                   [-0.3717, -0.9159, -0.9777],
│                   [-0.9135,  0.7139, -0.1748]])
└── 'x' --> <FastTreeValue 0x7f9090916250>
    └── 'c' --> tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                        [ 0.2954,  0.3656,  0.1234, -0.7910],
                 

In [23]:
%timeit t_cat(trees)

44.8 µs ± 3.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

86.1 µs ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2928,  1.4587, -1.0161, -1.8239],
                       [ 0.2954,  0.3656,  0.1234, -0.7910],
                       [-1.0263,  0.5554,  0.3729,  0.0998]],
              
                      [[ 0.2928,  1.4587, -1.0161, -1.8239],
                       [ 0.2954,  0.3656,  0.1234, -0.7910],
                       [-1.0263,  0.5554,  0.3729,  0.0998]],
              
                      [[ 0.2928,  1.4587, -1.0161, -1.8239],
                       [ 0.2954,  0.3656,  0.1234, -0.7910],
                       [-1.0263,  0.5554,  0.3729,  0.0998]],
              
                      [[ 0.2928,  1.4587, -1.0161, -1.8239],
                       [ 0.2954,  0.3656,  0.1234, -0.7910],
                       [-1.0263,  0.5554,  0.3729,  0.0998]],
              
                      [[ 0.2928,  1.4587, -1.0161, -1.8239],
                       [ 0.2954,  0.3656,  0.1234, -0.7910],
                       [-1.0263,  0.5554,  0.3729,  0.0998]],

In [26]:
%timeit Batch.stack(batches)

103 µs ± 4.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998],
                      [ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998],
                      [ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998],
                      [ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998],
                      [ 0.2928,  1.4587, -1.0161, -1.8239],
                      [ 0.2954,  0.3656,  0.1234, -0.7910],
                      [-1.0263,  0.5554,  0.3729,  0.0998],
                      [ 0.2928,  1.4587, -1.0161, -1.8239],
                   

In [28]:
%timeit Batch.cat(batches)

187 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

606 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
